In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Min-Max scaling
def MinMaxScaler(data):
    numerator = data - np.min(data, axis=0)
    denominator = np.max(data, axis=0) - np.min(data, axis=0)
    return numerator / (denominator + 1e-7)

# load data
stock = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
# change time-series order
stock = stock[::-1]
# applying scaling
stock = MinMaxScaler(stock)


### Train Parameter ###
# input : 7 days
seq_length = 7
# Feature(including Y)
input_units = 5
# Hidden units in RNN Cells
hidden_units = 10
# output : future 2 consecutive days
output_units = 2
learning_rate = 0.01
iterations = 10001

# Feature(including 2 pre-order Y variables)
x = stock
y = stock[:, [-1]]

# Build dataset for predicting two-consecutive time-series
dataX = []
dataY = []
cut_num = len(y) // 2

# x는 730번 인덱스까지(총 732번 인덱스까지 중)
for i in range(0, cut_num):
    if i <= 362:
        #print(i*2, i*2+seq_length-1)
        #print(i*2+seq_length, i*2+seq_length+1)
        #print()
        _x = x[i*2: i*2+seq_length]
        _y = y[i*2+seq_length: i*2+seq_length+2]
        dataX.append(_x)
        dataY.append(_y)
        
# split train/test
train_size = int(len(dataY) *0.7)
test_size = len(dataY) - train_size

trainX = np.array(dataX[0:train_size])
testX = np.array(dataX[train_size:len(dataX)-1])
trainY = np.array(dataY[0:train_size]).reshape(-1,2)
testY = np.stack(dataY[train_size:len(dataY)-1]).reshape(-1,2)

#print("X shape: ", trainX.shape, testX.shape)
#print("Y shape: ", trainY.shape, testY.shape)


tf.reset_default_graph()
tf.set_random_seed(123)

#### Build Tensor Graph ####
X = tf.placeholder(tf.float32, shape=[None, seq_length, input_units])
Y = tf.placeholder(tf.float32, shape=[None, output_units])

# Basic RNN Cell
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_units,
                                  activation=tf.tanh)

# Outputs in RNN Cell(hidden units)
outputs_seq, states = tf.nn.dynamic_rnn(cell, inputs=X,
                                       dtype=tf.float32)

# hidden units -> Output units 로가는 Dense layer
Y_pred = tf.contrib.layers.fully_connected(outputs_seq[:,-1],
                                          output_units,
                                          activation_fn=None)

# Cost function - Sum of Squared error
cost = tf.reduce_sum(tf.square(Y_pred - Y))

# Optimizer
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#### Run Tensor Graph ####
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 50번의 Epoch 수행
    for i in range(iterations):
        _ = sess.run(train, feed_dict={X : trainX,
                                      Y: trainY})
        if i % 100 == 0:
            cost_val = sess.run(cost, feed_dict={X: trainX,
                                                Y: trainY})
            print(f"Epoch{i}번일 때, Cost:{cost_val :.3f}")
    print("-"*10,"Learning finished","-"*10)
    # Test 데이터로 검증 후  Mean of Squared Error 도출
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    print(test_predict)
    mse = np.mean(np.square(test_predict - testY))
    print(f"RMSE for Test data: {mse :.3f}")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch0번일 때, Cost:47.349
Epoch100번일 때, Cost:0.775
Epoch200번일 때, Cost:0.707
Epoch300번일 때, Cost:0.672
Epoch400번일 때, Cost:0.653
Epoch500번일 때, Cost:0.638
Epoch600번일 때, Cost:0.

---

## 삽질 코드

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Min-Max scaling
def MinMaxScaler(data):
    numerator = data - np.min(data, axis=0)
    denominator = np.max(data, axis=0) - np.min(data, axis=0)
    return numerator / (denominator + 1e-7)

In [4]:
# load data
stock = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
# change time-series order
stock = stock[::-1]
# applying scaling
stock = MinMaxScaler(stock)

# Feature(including 2 pre-order Y variables)
x = stock
y = stock[:, [-1]]

In [5]:
### Train Parameter ###

# input : 7 days
seq_length = 7
# Feature(including Y)
input_units = 5
# Hidden units in RNN Cells
hidden_units = 10
# output : future 2 consecutive days
output_units = 2
learning_rate = 0.01
iterations = 10001

In [6]:
dataX = []
dataY = []
cut_num = len(y) // 2

# x는 730번 인덱스까지(총 732번 인덱스까지 중)
for i in range(0, cut_num):
    if i <= 362:
        print(i*2, i*2+seq_length-1)
        print(i*2+seq_length, i*2+seq_length+1)
        print()
        _x = x[i*2: i*2+seq_length]
        _y = y[i*2+seq_length: i*2+seq_length+2]
        dataX.append(_x)
        dataY.append(_y)

0 6
7 8

2 8
9 10

4 10
11 12

6 12
13 14

8 14
15 16

10 16
17 18

12 18
19 20

14 20
21 22

16 22
23 24

18 24
25 26

20 26
27 28

22 28
29 30

24 30
31 32

26 32
33 34

28 34
35 36

30 36
37 38

32 38
39 40

34 40
41 42

36 42
43 44

38 44
45 46

40 46
47 48

42 48
49 50

44 50
51 52

46 52
53 54

48 54
55 56

50 56
57 58

52 58
59 60

54 60
61 62

56 62
63 64

58 64
65 66

60 66
67 68

62 68
69 70

64 70
71 72

66 72
73 74

68 74
75 76

70 76
77 78

72 78
79 80

74 80
81 82

76 82
83 84

78 84
85 86

80 86
87 88

82 88
89 90

84 90
91 92

86 92
93 94

88 94
95 96

90 96
97 98

92 98
99 100

94 100
101 102

96 102
103 104

98 104
105 106

100 106
107 108

102 108
109 110

104 110
111 112

106 112
113 114

108 114
115 116

110 116
117 118

112 118
119 120

114 120
121 122

116 122
123 124

118 124
125 126

120 126
127 128

122 128
129 130

124 130
131 132

126 132
133 134

128 134
135 136

130 136
137 138

132 138
139 140

134 140
141 142

136 142
143 144

138 144
145 146

140 146
14

In [13]:
test_size + train_size == len(dataX)

True

In [14]:
test_size

109

In [27]:
# split train/test
train_size = int(len(dataY) *0.7)
test_size = len(dataY) - train_size

trainX = np.array(dataX[0:train_size])
testX = np.array(dataX[train_size:len(dataX)]-7)
trainY = np.array(dataY[0:train_size]).reshape(-1,2)
testY = np.stack(dataY[train_size:len(dataY)-1]).reshape(-1,2)

TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [9]:


'''
# 끝에서 데이터 하나 빼주었음..shape 안맞음..왜 안맞지..?
trainX = np.array(dataX[0:train_size])
testX = np.array(dataX[train_size:len(dataX)])
trainY = np.array(dataY[0:train_size]).reshape(-1,2)
testY = np.stack(dataY[train_size:len(dataY)]).reshape(-1,2)
'''

#print("X shape: ", trainX.shape, testX.shape)
#print("Y shape: ", trainY.shape, testY.shape)

'\n# 끝에서 데이터 하나 빼주었음..shape 안맞음..왜 안맞지..?\ntrainX = np.array(dataX[0:train_size])\ntestX = np.array(dataX[train_size:len(dataX)])\ntrainY = np.array(dataY[0:train_size]).reshape(-1,2)\ntestY = np.stack(dataY[train_size:len(dataY)]).reshape(-1,2)\n'

In [52]:
tf.reset_default_graph()
tf.set_random_seed(123)

X = tf.placeholder(tf.float32, shape=[None, seq_length, input_units])
Y = tf.placeholder(tf.float32, shape=[None, output_units])

# Basic RNN Cell
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_units,
                                  activation=tf.tanh)

# Outputs in RNN Cell(hidden units)
outputs_seq, states = tf.nn.dynamic_rnn(cell, inputs=X,
                                       dtype=tf.float32)

# hidden units -> Output units 로가는 Dense layer
Y_pred = tf.contrib.layers.fully_connected(outputs_seq[:,-1],
                                          output_units,
                                          activation_fn=None)

# Cost function - Sum of Squared error
cost = tf.reduce_sum(tf.square(Y_pred - Y))

# Optimizer
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#### Run Tensor Graph ####
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 50번의 Epoch 수행
    for i in range(iterations):
        _ = sess.run(train, feed_dict={X : trainX,
                                      Y: trainY})
        if i % 100 == 0:
            cost_val = sess.run(cost, feed_dict={X: trainX,
                                                Y: trainY})
            print(f"Epoch{i}번일 때, Cost:{cost_val :.3f}")
    print("-"*10,"Learning finished","-"*10)
    # Test 데이터로 검증 후  Mean of Squared Error 도출
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    mse = np.mean(np.square(test_predict - testY))
    print(f"RMSE for Test data: {mse :.3f}")
    

Epoch0번일 때, Cost:47.349
Epoch100번일 때, Cost:0.775
Epoch200번일 때, Cost:0.707
Epoch300번일 때, Cost:0.672
Epoch400번일 때, Cost:0.653
Epoch500번일 때, Cost:0.638
Epoch600번일 때, Cost:0.623
Epoch700번일 때, Cost:0.610
Epoch800번일 때, Cost:0.600
Epoch900번일 때, Cost:0.589
Epoch1000번일 때, Cost:0.589
Epoch1100번일 때, Cost:0.572
Epoch1200번일 때, Cost:0.565
Epoch1300번일 때, Cost:0.559
Epoch1400번일 때, Cost:0.735
Epoch1500번일 때, Cost:0.547
Epoch1600번일 때, Cost:0.541
Epoch1700번일 때, Cost:0.529
Epoch1800번일 때, Cost:0.519
Epoch1900번일 때, Cost:0.560
Epoch2000번일 때, Cost:0.509
Epoch2100번일 때, Cost:0.490
Epoch2200번일 때, Cost:0.481
Epoch2300번일 때, Cost:0.473
Epoch2400번일 때, Cost:0.464
Epoch2500번일 때, Cost:0.471
Epoch2600번일 때, Cost:0.451
Epoch2700번일 때, Cost:0.447
Epoch2800번일 때, Cost:0.444
Epoch2900번일 때, Cost:0.441
Epoch3000번일 때, Cost:0.443
Epoch3100번일 때, Cost:0.437
Epoch3200번일 때, Cost:0.432
Epoch3300번일 때, Cost:0.470
Epoch3400번일 때, Cost:0.462
Epoch3500번일 때, Cost:0.525
Epoch3600번일 때, Cost:0.431
Epoch3700번일 때, Cost:0.410
Epoch3800번일 때, Cost:0.4